In [17]:
import pandas as pd
from lxml import etree
import requests

In [26]:
url = "https://www.hemnet.se/bostader/bbox"

params={
    'bbox':'59.247127100450335,17.76513093234948:59.35859632184681,18.403711254615104',
    'search_key':'57938136258b743b4bfb313c013169b07ef4cffd'
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

payload = {}

In [27]:
response = requests.request("GET", url, headers=headers, data=payload, params=params)

j=response.json()['properties']

In [20]:
df=pd.json_normalize(j, max_level=1)
#df.keys
df.head(3)

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,...,broker_agency_name,locations_string,should_display_showings,should_highlight_showings,living_space.table,trackingData.listingId,trackingData.listingType,trackingData.listingPackageType,trackingData.listingState,trackingData.listingStatus
0,19226516,"[59.3188273746, 18.0518373102]",Ringvägen 9 F,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,Fastighetsbyrån Stockholm - Södermalm,"Högalid, Stockholms kommun",False,False,"{'living_area': '74 m²', 'supplemental_area': ...",19226516,common_listing,basic,active,for_sale
1,19226416,"[59.3085175, 18.0801334]",Ringvägen 135,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,HusmanHagberg Södermalm/Gamla Stan,"Södermalm - SoFo, Stockholms kommun",False,True,"{'living_area': '42 m²', 'supplemental_area': ...",19226416,common_listing,premium,active,for_sale
2,19226122,"[59.3094732, 18.083821]","Ringvägen 153, 2 tr",Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,Mäklarhuset Stockholm Innerstan,"Stockholm - Södermalm, Stockholms kommun",False,False,"{'living_area': '42 m²', 'supplemental_area': ...",19226122,common_listing,basic,active,for_sale


In [21]:
#call graphql directly from listing with ids - example with one id only but can set into a loop
ql_id=df.iloc[1]['id']
ql_id

19226416

In [22]:
url_ql = "https://www.hemnet.se/graphql"

#would love to have payload_ql string formatted better but not really sure how without breaking it
payload_ql="{\"query\":\"query imagesForListing($id: ID!) {\\r\\n    listing(id: $id) \\r\\n    {  id  __typename  streetAddress  isSaved  brokerAgency \\r\\n    {    name    id    brokerCustomization \\r\\n    {      compactLogoUrl      largeLogoUrl: logoUrl(format: BROKER_CUSTOMIZATION_LARGE)      __typename    \\r\\n    }    __typename  }  ... on ActivePropertyListing \\r\\n    {    liveStreams(scope: ENDED) \\r\\n    {      embedUrl      __typename    \\r\\n    }    videoAttachment: attachment(type: VIDEO) \\r\\n    {      id      attachmentType      ... on VideoAttachment {     videoHemnetUrl        __typename     }\\r\\n          __typename    \\r\\n    }    threeDAttachment: attachment(type: THREE_D) {\\r\\n              id      ... on ThreeDAttachment {\\r\\n                          url        __typename      \\r\\n                          }      __typename    }\\r\\n                              isForeclosure    listingBrokerGalleryUrl    images(limit: 500) {\\r\\n                                        images {\\r\\n                                                    url(format: ITEMGALLERY_CUT)        portraitUrl: url(format: ITEMGALLERY_PORTRAIT_CUT)        fullscreenUrl: url(format: WIDTH1024)        originalHeight        originalWidth        labels        __typename      }      __typename    }    __typename  }  ... on ProjectUnit {    liveStreams(scope: ENDED) {      embedUrl      __typename    }    videoAttachment: attachment(type: VIDEO) {      id      attachmentType      ... on VideoAttachment {        videoHemnetUrl        __typename      }      __typename    }    threeDAttachment: attachment(type: THREE_D) {      id      ... on ThreeDAttachment {        url        __typename      }      __typename    }    isForeclosure    listingBrokerGalleryUrl    images(limit: 500) {      images {        url(format: ITEMGALLERY_CUT)        portraitUrl: url(format: ITEMGALLERY_PORTRAIT_CUT)        fullscreenUrl: url(format: WIDTH1024)        originalHeight        originalWidth        labels        __typename      }      __typename    }    __typename  }\\r\\n    }\\r\\n    }\",\"variables\":{\"id\":"+str(ql_id)+"}}"

headers_ql = {
  'Content-Type': 'application/json',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response_ql = requests.request("POST", url_ql, headers=headers_ql, data=payload_ql)

#print(response_ql.text)
ql_path=response_ql.json()['data']['listing']['images']['images']

ql_df=pd.json_normalize(ql_path)
#ql_df

In [23]:
#print all images from listing item
from IPython.display import Image
for index,row in ql_df.iterrows():
   display(Image(url=row['url']))